In [1]:
import open3d as o3d
import trimesh
import pyrender
import matplotlib.pyplot as plt
import numpy as np

INFO - 2020-11-02 16:58:14,466 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [16]:
def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec.T


num_samples = 100000
theta = np.random.rand(num_samples) * np.pi
phi = np.random.rand(num_samples) * 2 * np.pi
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

pcd = trimesh.PointCloud(
    vertices=[
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]
    ]
)
# pcd = o3d.geometry.PointCloud(points=o3d.open3d_pybind.utility.Vector3dVector(np.array([
#     [0, 0, 0],
#     [0, 0, 1],
#     [0, 1, 0],
#     [0, 1, 1],
#     [1, 0, 0],
#     [1, 0, 1],
#     [1, 1, 0],
#     [1, 1, 1]
# ])))


# pcd = o3d.geometry.PointCloud(points=o3d.open3d_pybind.utility.Vector3dVector(
#     sample_spherical(num_samples)
# #     np.stack([x, y, z], axis=-1)
# ))
# # 


# pcd.estimate_normals()

# estimate radius for rolling ball
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 1.5 * avg_dist   

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
           pcd,
           o3d.utility.DoubleVector([radius, radius * 2]))

# create the triangular mesh with the vertices and faces from open3d
tri_mesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles),
                          vertex_normals=np.asarray(mesh.vertex_normals))

trimesh.convex.is_convex(tri_mesh)

AttributeError: 'PointCloud' object has no attribute 'compute_nearest_neighbor_distance'

In [6]:
tri_mesh = trimesh.primitives.Box()
# tri_mesh = trimesh.primitives.Sphere()

In [8]:
pyrender_mesh = pyrender.Mesh.from_trimesh(tri_mesh)
scene = pyrender.Scene()
scene.add(pyrender_mesh)
pyrender.Viewer(scene, use_raymond_lighting=True)

Viewer(width=640, height=480)

In [9]:
fuze_trimesh = trimesh.load('fuze.obj')


mesh = pyrender.Mesh.from_trimesh(fuze_trimesh)
scene = pyrender.Scene()
scene.add(mesh)
pyrender.Viewer(scene, use_raymond_lighting=True)

ValueError: string is not a file: fuze.obj

In [10]:
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
s = np.sqrt(2)/2
camera_pose = np.array([
    [0.0, -s,   s,   0.3],
   [1.0,  0.0, 0.0, 0.0],
    [0.0,  s,   s,   0.35],
    [0.0,  0.0, 0.0, 1.0],
])
scene.add(camera, pose=camera_pose)
light = pyrender.SpotLight(color=np.ones(3), intensity=3.0,
                           innerConeAngle=np.pi/16.0,
                           outerConeAngle=np.pi/6.0)
scene.add(light, pose=camera_pose)
r = pyrender.OffscreenRenderer(400, 400)
color, depth = r.render(scene)


AttributeError: module 'pyglet.gl' has no attribute 'xlib'